## Prever Sobreviventes no Titanic (classificação binária)

In [1]:
import pandas as pd
import numpy as np

# Ler dados do arquivo CSV para um DataFrame pandas
file_path = 'C:/Users/karin/OneDrive/Área de Trabalho/data science/titanic/train.csv'
df_titanic = pd.read_csv(file_path)

df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Analisar modelos

### Importar bibliotecas

In [2]:
# Importando bibliotecas necessárias

# Para manipular os dados
import pandas as pd 

# Pré-processamento
from sklearn.impute import SimpleImputer  # Para imputação de valores ausentes
from sklearn.preprocessing import OneHotEncoder  # Para codificar variáveis categóricas
from sklearn.preprocessing import LabelEncoder # Para codificar variáveis categóricas
from sklearn.preprocessing import StandardScaler # Para padronizar os dados
from sklearn.preprocessing import MinMaxScaler # Para normalizar os dados

# Treinamento com Múltiplos Modelos de Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

# Seleção de Modelo
from sklearn.model_selection import train_test_split # Para dividir o dataset em treino e teste
from sklearn.model_selection import GridSearchCV # Mudar os argumentos dos modelos em busca de otimização(Hiperparameter tuning)

# Avaliação de resultados
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

# Para criar um pipeline
from sklearn.pipeline import Pipeline

# Biblioteca para acessar o Feature Store
from databricks.feature_store import FeatureStoreClient

# Para registrar o modelo
import mlflow
mlflow.set_registry_uri("databricks-uc")

import warnings # Para ignorar avisos
warnings.filterwarnings('ignore')

## (Extrair) Ingestão de dados
https://code.datasciencedojo.com/datasciencedojo/datasets

In [9]:
import duckdb

# Carregando o dataset Titanic
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
dados_titanic = pd.read_csv(url)

# Renomear as colunas para português
dados_titanic.rename(columns={
    'PassengerId': 'IdPassageiro',
    'Survived': 'Sobreviveu',
    'Pclass': 'Classe',
    'Name': 'Nome',
    'Sex': 'Sexo',
    'Age': 'Idade',
    'SibSp': 'IrmaosConjuges',
    'Parch': 'PaisFilhos',
    'Ticket': 'Bilhete',
    'Fare': 'Tarifa',
    'Cabin': 'Cabine',
    'Embarked': 'Embarque'
}, inplace=True)

# este comando só funciona no databricks. Se necessário, use head() para visualizar as primeiras linhas
display(dados_titanic)
# dados.head()

,IdPassageiro,Sobreviveu,Classe,Nome,Sexo,Idade,IrmaosConjuges,PaisFilhos,Bilhete,Tarifa,Cabine,Embarque
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## (Carga) Registro no Unity Catalog

In [10]:
# codigo no databricks
# dados_titanic_spark = spark.createDataFrame(dados_titanic)
# dados_titanic_spark.write.format('delta').mode('overwrite').saveAsTable('coderhouse_ds_58690.default.tabela_titanic')

# O código abaixo adapta para criar uma tabela em DuckDB, salvando os dados em um arquivo de banco DuckDB em vez de um catálogo distribuído como o Unity Catalog.
# Conectar ao banco de dados DuckDB (pode ser em memória ou em arquivo)
con = duckdb.connect('titanic.duckdb')  # Salva no arquivo 'titanic.duckdb'

# Criar uma tabela no DuckDB
con.execute("CREATE OR REPLACE TABLE coderhouse_ds_58690_default_tabela_titanic AS SELECT * FROM dados_titanic")

# Verificar o conteúdo da tabela no DuckDB
result = con.execute("SELECT * FROM coderhouse_ds_58690_default_tabela_titanic LIMIT 5").df()
print(result)

# Fechar a conexão
con.close()

   IdPassageiro  Sobreviveu  Classe  \
0             1           0       3   
1             2           1       1   
2             3           1       3   
3             4           1       1   
4             5           0       3   

                                                Nome    Sexo  Idade  \
0                            Braund, Mr. Owen Harris    male   22.0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female   38.0   
2                             Heikkinen, Miss. Laina  female   26.0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female   35.0   
4                           Allen, Mr. William Henry    male   35.0   

   IrmaosConjuges  PaisFilhos           Bilhete   Tarifa Cabine Embarque  
0               1           0         A/5 21171   7.2500   None        S  
1               1           0          PC 17599  71.2833    C85        C  
2               0           0  STON/O2. 3101282   7.9250   None        S  
3               1           0         

## Tratamento e Pré-processamento

In [11]:
# Verificar valores ausentes
dados_titanic.isnull().sum()

IdPassageiro        0
Sobreviveu          0
Classe              0
Nome                0
Sexo                0
Idade             177
IrmaosConjuges      0
PaisFilhos          0
Bilhete             0
Tarifa              0
Cabine            687
Embarque            2
dtype: int64

In [12]:
dados_titanic.head()

,IdPassageiro,Sobreviveu,Classe,Nome,Sexo,Idade,IrmaosConjuges,PaisFilhos,Bilhete,Tarifa,Cabine,Embarque
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
dados_titanic.Embarque.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [15]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Exemplo de uso do OneHotEncoder para transformar variáveis categóricas
encoder = OneHotEncoder(sparse_output=False)  # Atualizado para sparse_output
dados_categoricos = ['Sexo', 'Embarque']
dados_transformados = encoder.fit_transform(dados_titanic[dados_categoricos])

# Converter o array numpy de volta para um DataFrame para melhor visualização
colunas_transformadas = encoder.get_feature_names_out(dados_categoricos)
dados_transformados_df = pd.DataFrame(dados_transformados, columns=colunas_transformadas)

display(dados_transformados_df)


,Sexo_female,Sexo_male,Embarque_C,Embarque_Q,Embarque_S,Embarque_nan
0,0.0,1.0,0.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
886,0.0,1.0,0.0,0.0,1.0,0.0
887,1.0,0.0,0.0,0.0,1.0,0.0
888,1.0,0.0,0.0,0.0,1.0,0.0
889,0.0,1.0,1.0,0.0,0.0,0.0


In [16]:
dados = dados_titanic.copy()
# NOVA LINHAS ENTRAM AQUI

# Remover colunas irrelevantes
dados.drop(['Nome', 'Bilhete', 'Cabine'], axis=1, inplace=True)

# Preencher valores ausentes em 'Idade' com a mediana
imputador = SimpleImputer(strategy='median') # Pode ser 'median', 'most_frequent', 'constant', etc.
dados['Idade'] = imputador.fit_transform(dados[['Idade']])

# Preencher valores ausentes em 'Embarque' com a moda
dados['Embarque'].fillna(dados['Embarque'].mode()[0], inplace=True)

# Feature Engineering
dados['TamanhoFamilia'] = dados['IrmaosConjuges'] + dados['PaisFilhos']
dados['EstaSozinho'] = (dados['TamanhoFamilia'] == 0).astype(int)

# Codificação numérica de variáveis categóricas
encoder = LabelEncoder()
dados['Sexo_Codificado'] = encoder.fit_transform(dados['Sexo'])
dados['Embarque_Codificado'] = encoder.fit_transform(dados['Embarque'])

# Remover colunas não mais necessárias
dados.drop(['Sexo', 'Embarque'], axis=1, inplace=True)

# Escalonar características numéricas (Padronizar e Normalizar)
caracteristicas = ['Idade', 'Tarifa', 'IrmaosConjuges', 'PaisFilhos', 'TamanhoFamilia']

# Padronizando os dados
scaler_standard = StandardScaler()
dados[caracteristicas] = scaler_standard.fit_transform(dados[caracteristicas])

# Normalizando os dados
scaler_minmax = MinMaxScaler()
dados[caracteristicas] = scaler_minmax.fit_transform(dados[caracteristicas])

dados_preprocessados = dados

# AQUI TERMINA A TRANSFORMAÇÃO
dados.head()

,IdPassageiro,Sobreviveu,Classe,Idade,IrmaosConjuges,PaisFilhos,Tarifa,TamanhoFamilia,EstaSozinho,Sexo_Codificado,Embarque_Codificado
0,1,0,3,0.271174,0.125,0.0,0.014151,0.1,0,1,2
1,2,1,1,0.472229,0.125,0.0,0.139136,0.1,0,0,0
2,3,1,3,0.321438,0.000,0.0,0.015469,0.0,1,0,2
3,4,1,1,0.434531,0.125,0.0,0.103644,0.1,0,0,2
4,5,0,3,0.434531,0.000,0.0,0.015713,0.0,1,1,2


In [17]:
# Verificar valores ausentes
dados.isnull().sum()

IdPassageiro           0
Sobreviveu             0
Classe                 0
Idade                  0
IrmaosConjuges         0
PaisFilhos             0
Tarifa                 0
TamanhoFamilia         0
EstaSozinho            0
Sexo_Codificado        0
Embarque_Codificado    0
dtype: int64

In [18]:
# Realizar o pré-processamento e criar features
def preprocess_data(dados):
    # Copiar o DataFrame
    dados = dados_titanic.copy() 

    # Remover colunas irrelevantes
    dados.drop(['Nome', 'Bilhete', 'Cabine'], axis=1, inplace=True)

    # Preencher valores ausentes em 'Idade' com a mediana
    imputador = SimpleImputer(strategy='median') # Pode ser 'median', 'most_frequent', 'constant', etc.
    dados['Idade'] = imputador.fit_transform(dados[['Idade']])

    # Preencher valores ausentes em 'Embarque' com a moda
    dados['Embarque'].fillna(dados['Embarque'].mode()[0], inplace=True)

    # Feature Engineering
    dados['TamanhoFamilia'] = dados['IrmaosConjuges'] + dados['PaisFilhos']
    dados['EstaSozinho'] = (dados['TamanhoFamilia'] == 0).astype(int)

    # Codificação numérica de variáveis categóricas
    encoder = LabelEncoder()
    dados['Sexo_Codificado'] = encoder.fit_transform(dados['Sexo'])
    dados['Embarque_Codificado'] = encoder.fit_transform(dados['Embarque'])

    # Remover colunas não mais necessárias
    dados.drop(['Sexo', 'Embarque'], axis=1, inplace=True)

    # Escalonar características numéricas (Padronizar e Normalizar)
    caracteristicas = ['Idade', 'Tarifa', 'IrmaosConjuges', 'PaisFilhos', 'TamanhoFamilia']

    # Padronizando os dados
    scaler_standard = StandardScaler()
    dados[caracteristicas] = scaler_standard.fit_transform(dados[caracteristicas])

    # Normalizando os dados
    scaler_minmax = MinMaxScaler()
    dados[caracteristicas] = scaler_minmax.fit_transform(dados[caracteristicas])

    return dados

dados_preprocessados = preprocess_data(dados_titanic)

# Este comando só funciona no databricks.
# Se necessário, use head() para visualizar as primeiras linhas
display(dados_preprocessados)
# dados.head()

,IdPassageiro,Sobreviveu,Classe,Idade,IrmaosConjuges,PaisFilhos,Tarifa,TamanhoFamilia,EstaSozinho,Sexo_Codificado,Embarque_Codificado
0,1,0,3,0.271174,0.125,0.000000,0.014151,0.1,0,1,2
1,2,1,1,0.472229,0.125,0.000000,0.139136,0.1,0,0,0
2,3,1,3,0.321438,0.000,0.000000,0.015469,0.0,1,0,2
3,4,1,1,0.434531,0.125,0.000000,0.103644,0.1,0,0,2
4,5,0,3,0.434531,0.000,0.000000,0.015713,0.0,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,0.334004,0.000,0.000000,0.025374,0.0,1,1,2
887,888,1,1,0.233476,0.000,0.000000,0.058556,0.0,1,0,2
888,889,0,3,0.346569,0.125,0.333333,0.045771,0.3,0,0,2
889,890,1,1,0.321438,0.000,0.000000,0.058556,0.0,1,1,0


In [19]:
# Verificar novamente valores ausentes
dados_preprocessados.isnull().sum()

IdPassageiro           0
Sobreviveu             0
Classe                 0
Idade                  0
IrmaosConjuges         0
PaisFilhos             0
Tarifa                 0
TamanhoFamilia         0
EstaSozinho            0
Sexo_Codificado        0
Embarque_Codificado    0
dtype: int64

## Criação de Features com o Feature Store

In [20]:
import duckdb
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

# Carregando o dataset Titanic
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
dados_titanic = pd.read_csv(url)

# Realizar o pré-processamento e criar features
def preprocess_data(dados):
    # Copiar o DataFrame
    dados = dados.copy()

    # Remover colunas irrelevantes
    dados.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

    # Preencher valores ausentes em 'Age' com a mediana
    imputador = SimpleImputer(strategy='median')
    dados['Age'] = imputador.fit_transform(dados[['Age']])

    # Preencher valores ausentes em 'Embarked' com a moda
    dados['Embarked'].fillna(dados['Embarked'].mode()[0], inplace=True)

    # Feature Engineering: criar novas colunas
    dados['FamilySize'] = dados['SibSp'] + dados['Parch']
    dados['IsAlone'] = (dados['FamilySize'] == 0).astype(int)

    # Codificação numérica de variáveis categóricas
    encoder = LabelEncoder()
    dados['Sex_Coded'] = encoder.fit_transform(dados['Sex'])
    dados['Embarked_Coded'] = encoder.fit_transform(dados['Embarked'])

    # Remover colunas não mais necessárias
    dados.drop(['Sex', 'Embarked'], axis=1, inplace=True)

    # Escalonar características numéricas (Padronizar e Normalizar)
    features_to_scale = ['Age', 'Fare', 'SibSp', 'Parch', 'FamilySize']

    # Padronizando os dados
    scaler_standard = StandardScaler()
    dados[features_to_scale] = scaler_standard.fit_transform(dados[features_to_scale])

    # Normalizando os dados
    scaler_minmax = MinMaxScaler()
    dados[features_to_scale] = scaler_minmax.fit_transform(dados[features_to_scale])

    return dados

# Aplicar o pré-processamento ao dataset Titanic
dados_preprocessados = preprocess_data(dados_titanic)

# Conectar ao banco de dados DuckDB
con = duckdb.connect('features_titanic.duckdb')

# Criar a tabela de features no DuckDB com chave primária
con.execute("""
CREATE OR REPLACE TABLE coderhouse_ds_58690_default_features_titanic AS 
SELECT * FROM dados_preprocessados
""")

# Definir chave primária (IdPassageiro)
con.execute("""
CREATE UNIQUE INDEX IF NOT EXISTS idx_features_titanic_idpassageiro 
ON coderhouse_ds_58690_default_features_titanic(PassengerId)
""")

# Simular os metadados (tags) como uma tabela separada no DuckDB
tags = {
    'dataset': 'titanic',
    'projeto': 'prod_gbs_finance_latam',
    'estagio': 'teste',
    'linguagem': 'python',
    'framework': 'duckdb',
    'linguagem_dados': 'pandas',
    'data_carga': '2023-01-01',
    'carga_automatica': 'nao',
    'carga_manual': 'sim',
    'tipo': 'feature_store'
}

# Convertendo os metadados em um DataFrame para armazená-los no DuckDB
tags_df = pd.DataFrame(list(tags.items()), columns=['tag', 'valor'])

# Criar uma tabela de metadados (tags) no DuckDB
con.execute("CREATE OR REPLACE TABLE features_titanic_metadata AS SELECT * FROM tags_df")

# Verificar o conteúdo das tabelas criadas
features_result = con.execute("SELECT * FROM coderhouse_ds_58690_default_features_titanic LIMIT 5").df()
metadata_result = con.execute("SELECT * FROM features_titanic_metadata").df()

# Mostrar os dados da tabela de features e de metadados
print("Tabela de Features (Titanic):")
print(features_result)

print("\nTabela de Metadados (Tags):")
print(metadata_result)

# Fechar a conexão
con.close()


Tabela de Features (Titanic):
   PassengerId  Survived  Pclass       Age  SibSp  Parch      Fare  \
0            1         0       3  0.271174  0.125    0.0  0.014151   
1            2         1       1  0.472229  0.125    0.0  0.139136   
2            3         1       3  0.321438  0.000    0.0  0.015469   
3            4         1       1  0.434531  0.125    0.0  0.103644   
4            5         0       3  0.434531  0.000    0.0  0.015713   

   FamilySize  IsAlone  Sex_Coded  Embarked_Coded  
0         0.1        0          1               2  
1         0.1        0          0               0  
2         0.0        1          0               2  
3         0.1        0          0               2  
4         0.0        1          1               2  

Tabela de Metadados (Tags):
                tag                   valor
0           dataset                 titanic
1           projeto  prod_gbs_finance_latam
2           estagio                   teste
3         linguagem             

## Divisão dos Dados (Data Splitting)
Dividimos os dados em dois grupos:

Dados de Treinamento: Usados para ensinar o modelo a identificar padrões.
Dados de Teste: Usados para verificar se o modelo aprendeu bem e faz previsões corretas em situações novas.
Essa divisão é importante para garantir que o modelo não apenas memorize os exemplos, mas realmente aprenda a generalizar para dados que nunca viu antes. Assim, asseguramos que ele funcionará bem no mundo real.

In [21]:
dados_preprocessados.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize,IsAlone,Sex_Coded,Embarked_Coded
0,1,0,3,0.271174,0.125,0.0,0.014151,0.1,0,1,2
1,2,1,1,0.472229,0.125,0.0,0.139136,0.1,0,0,0
2,3,1,3,0.321438,0.000,0.0,0.015469,0.0,1,0,2
3,4,1,1,0.434531,0.125,0.0,0.103644,0.1,0,0,2
4,5,0,3,0.434531,0.000,0.0,0.015713,0.0,1,1,2


In [23]:
# Usar os nomes corretos das colunas no DataFrame
X = dados_preprocessados.drop(['PassengerId', 'Survived'], axis=1)
X.head()


,Pclass,Age,SibSp,Parch,Fare,FamilySize,IsAlone,Sex_Coded,Embarked_Coded
0,3,0.271174,0.125,0.0,0.014151,0.1,0,1,2
1,1,0.472229,0.125,0.0,0.139136,0.1,0,0,0
2,3,0.321438,0.000,0.0,0.015469,0.0,1,0,2
3,1,0.434531,0.125,0.0,0.103644,0.1,0,0,2
4,3,0.434531,0.000,0.0,0.015713,0.0,1,1,2


In [24]:
y = dados_preprocessados['Survived']
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [25]:
# Separar features (caracteristicas) e target (alvo)

# Dividir em conjuntos de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [26]:
dados_preprocessados.shape

(891, 11)

In [27]:
X_treino.shape

(712, 9)

In [28]:
X_teste.shape

(179, 9)

## Teste de Vários Modelos
Lista de Modelos a Serem Testados.

Modelos de classificação binária
Regressão Logística
Árvore de Decisão
Random Forest
KNN
MLP
Máquina de Vetores de Suporte (SVM)
Gradient Boosting
XGBoost

In [29]:
# Dicionário para armazenar os resultados
modelos = {
    'Regressão Logística': LogisticRegression(),
    'Árvore de Decisão': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'MLP': MLPClassifier(max_iter=1000),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(probability=True),
    'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

In [30]:
modelos

{'Regressão Logística': LogisticRegression(),
 'Árvore de Decisão': DecisionTreeClassifier(),
 'KNN': KNeighborsClassifier(),
 'MLP': MLPClassifier(max_iter=1000),
 'Random Forest': RandomForestClassifier(),
 'SVM': SVC(probability=True),
 'Gradient Boosting': GradientBoostingClassifier(),
 'XGBoost': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric='logloss',
               feature_types=None, gamma=None, grow_policy=None,
               importance_type=None, interaction_constraints=None,
               learning_rate=None, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, multi_strategy=None, n_estimators=None,


In [32]:
# Lista para armazenar os resultados
resultados = []

# Avaliação de cada modelo
for nome_modelo, modelo in modelos.items():
    # Treinar o modelo
    modelo.fit(X_treino, y_treino)  # << aqui é executado o treinamento do modelo 

    # Previsões
    y_pred = modelo.predict(X_teste) # << aqui é feito a previsão diante de um modelo treinado

    # Acurácia
    acuracia = accuracy_score(y_teste, y_pred) # <<  comparar o que foi previsto com os dados que nos já sabemos que sao respostas corretas
    auc  = roc_auc_score(y_teste, modelo.predict_proba(X_teste)[:, 1]) # Prever as probabilidades no conjunto de teste

    # Adicionar os resultados a lista
    resultados.append({
        "Modelo": nome_modelo,
        "Acurácia": acuracia,
        "AUC-ROC": auc 
    })
    print(f'{nome_modelo}  - Acurácia: {acuracia:.2f}, AUC-ROC: {auc:.2f}')

Regressão Logística  - Acurácia: 0.80, AUC-ROC: 0.85
Árvore de Decisão  - Acurácia: 0.79, AUC-ROC: 0.77
KNN  - Acurácia: 0.79, AUC-ROC: 0.83
MLP  - Acurácia: 0.80, AUC-ROC: 0.86
Random Forest  - Acurácia: 0.82, AUC-ROC: 0.83
SVM  - Acurácia: 0.82, AUC-ROC: 0.81
Gradient Boosting  - Acurácia: 0.80, AUC-ROC: 0.82
XGBoost  - Acurácia: 0.80, AUC-ROC: 0.81


### Cálculo Simplificado do Accuracy Score (Acurácia)

O accuracy_score mede a proporção de previsões corretas feitas pelo modelo em relação ao total de previsões. É calculado assim:

Acurácia = (Número de Previsões Corretas) ÷ (Número Total de Previsões)
Exemplo:

Se o modelo fez 100 previsões e acertou 85 delas:

Acurácia = 85 ÷ 100 = 0,85 ou 85%
Resumo:

O accuracy_score indica a porcentagem de acertos do modelo.
Quanto mais próximo de 100%, melhor o desempenho em prever corretamente.

In [33]:
# Criar DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)
df_resultados['Combinada'] = df_resultados['Acurácia'] * df_resultados['AUC-ROC']

# Identificar o melhor modelo com base na métrica AUC
for valor in [ 'Acurácia', 'AUC-ROC', 'Combinada']:
    melhor_modelo_nome = df_resultados.sort_values(by=valor, ascending=False).iloc[0]['Modelo']
    print(f"Melhor modelo {valor}: {melhor_modelo_nome}")

df_resultados.sort_values(by='Acurácia', ascending=False)

Melhor modelo Acurácia: Random Forest
Melhor modelo AUC-ROC: MLP
Melhor modelo Combinada: MLP


,Modelo,Acurácia,AUC-ROC,Combinada
4,Random Forest,0.821229,0.830105,0.681707
5,SVM,0.815642,0.813834,0.663798
0,Regressão Logística,0.804469,0.854282,0.687244
6,Gradient Boosting,0.804469,0.819829,0.659527
3,MLP,0.798883,0.861397,0.688155
7,XGBoost,0.798883,0.812385,0.649000
1,Árvore de Decisão,0.793296,0.767918,0.609187
2,KNN,0.787709,0.832806,0.656009


In [35]:
# Identificar o melhor modelo com base na métrica AUC
melhor_modelo_nome = df_resultados.sort_values(by="Combinada", ascending=False).iloc[0]['Modelo']
print(f"Melhor modelo: {melhor_modelo_nome}")

Melhor modelo: MLP


## Otimização de Hiperparâmetros
(exemplo com Regressão Logística)

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Definir o modelo de Regressão Logística
regressao_logistica = LogisticRegression()

# Proposta de hiperparâmetros
param_grid = {
    'modelo__penalty': ['l1', 'l2', 'elasticnet', 'none'],  # Penalização (regularização)
    'modelo__C': [0.001, 0.01, 0.1, 1, 10, 100],            # Força de regularização (inverso da regularização)
    'modelo__solver': ['liblinear', 'saga', 'lbfgs'],       # Solver (otimizador)
    'modelo__fit_intercept': [True, False],                 # Incluir ou não o intercepto
    'modelo__max_iter': [100, 200, 500]                     # Número máximo de iterações
}

# Definir o pipeline com o pré-processador e o modelo
pipeline = Pipeline(steps=[('modelo', regressao_logistica)])

# Otimização de hiperparâmetros com GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

# Treinar o modelo com a busca em grade
grid_search.fit(X_treino, y_treino)

# Exibir os melhores hiperparâmetros e o melhor desempenho
print(f'Melhores parâmetros: {grid_search.best_params_}')
print(f'Melhor acurácia: {grid_search.best_score_:.4f}')

# cria modelo com os parâmetros mencionados
modelo = LogisticRegression(penalty='l1', C=0.001, solver='liblinear', fit_intercept=True, max_iter=100)

Melhores parâmetros: {'modelo__C': 1, 'modelo__fit_intercept': True, 'modelo__max_iter': 100, 'modelo__penalty': 'l1', 'modelo__solver': 'saga'}
Melhor acurácia: 0.7992


## Análise do Melhor Modelo

In [37]:
# Avaliação detalhada do Estimador (Modelo) de aprendizagem automática
modelo = LogisticRegression()

# Treinar o modelo
modelo.fit(X_treino, y_treino)

# Previsões
y_pred = modelo.predict(X_teste)

# Acurácia
acuracia = accuracy_score(y_teste, y_pred)
auc  = roc_auc_score(y_teste, modelo.predict_proba(X_teste)[:, 1]) # Prever as probabilidades no conjunto de teste

print(f'Regressão Logística  - Acurácia: {acuracia:.2f}, AUC-ROC: {auc:.2f}')

Regressão Logística  - Acurácia: 0.80, AUC-ROC: 0.85


## Experiment Tracking com MLflow

In [15]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

# Definir o caminho correto do experimento e nome do experimento
parent_folder = "C:/Users/karin/OneDrive/Área de Trabalho/data science/titanic"
nome_experimento = "experimento_titanic"
artifact_path_name = 'modelo_regressao_logistica'

# Configurar o URI de tracking para salvar os logs localmente
mlflow.set_tracking_uri(f"file:///{parent_folder}/mlflow_logs")

# Verificar se o experimento já existe, caso contrário, criar um novo
mlflow.set_experiment(nome_experimento)

# Nome do arquivo CSV com os dados do Titanic
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
dados_titanic = pd.read_csv(url)

# Preprocessamento do dataset Titanic
dados_titanic['TamanhoFamilia'] = dados_titanic['SibSp'] + dados_titanic['Parch']
dados_titanic['EstaSozinho'] = (dados_titanic['TamanhoFamilia'] == 0).astype(int)
dados_titanic['Sexo_Codificado'] = dados_titanic['Sex'].map({'male': 0, 'female': 1})
dados_titanic['Embarque_Codificado'] = dados_titanic['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# Definir os nomes das features
feature_names = [
    'Pclass', 'Age', 'Fare', 'SibSp', 'Parch',
    'TamanhoFamilia', 'EstaSozinho', 'Sexo_Codificado', 'Embarque_Codificado'
]

# Verificar se a coluna 'Survived' existe
assert 'Survived' in dados_titanic.columns, "A coluna 'Survived' não está presente no CSV."

# Remover valores nulos
dados_titanic = dados_titanic.dropna(subset=feature_names + ['Survived'])

# Separar features e target
X = dados_titanic[feature_names]
y = dados_titanic['Survived']

# Dividir os dados em conjuntos de treinamento e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_treino, y_treino)

# Previsões
y_pred = modelo.predict(X_teste)

# Acurácia
acuracia = accuracy_score(y_teste, y_pred)
auc_roc = roc_auc_score(y_teste, modelo.predict_proba(X_teste)[:, 1])

print(f'Regressão Logística - Acurácia: {acuracia:.2f}, AUC-ROC: {auc_roc:.2f}')

# Salvando métricas no MLflow
with mlflow.start_run(run_name="Regressão_Logística_Titanic"):
    mlflow.log_metric('acuracia', acuracia)
    mlflow.log_metric('auc_roc', auc_roc)
    mlflow.log_param('modelo', 'Regressão Logística')
    
    # Logar hiperparâmetros do modelo
    hiperparametros = modelo.get_params()
    mlflow.log_params(hiperparametros)

    # Salvar o modelo no MLflow
    mlflow.sklearn.log_model(modelo, artifact_path=artifact_path_name)

    run_id = mlflow.active_run().info.run_id
    model_uri = f"runs:/{run_id}/{artifact_path_name}"
    print(f"Modelo registrado no URI: {model_uri}")


ConnectionRefusedError: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente